# Patchy proteins potential 

Implempentation of two diffusing patchy particles (rigid bodies). One with 6 patches (one stronger than the others) and the second one with only one patch. The potenial based on patchy particles is defined in the code as pathcy protein potential since it is supposed to emulate protein-protein interaction. Once a binding is formed between atches, there is no preferred relative orientation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2
import msmrd2.tools.quaternions as quats
import msmrd2.visualization as msmrdvis
from msmrd2.potentials import patchyProtein
from msmrd2.integrators import overdampedLangevin as odLangevin

In [ ]:
# Define arbitrarily large list of patchy particles
numparticles = 2
D = 1.0
Drot = 1.0
pyPartlist = [] 
positionList = [np.array([1.0,0,0]), np.array([-1.,0.,0.])]
orientationList = [ np.array([1.,0.,0.,0.]), np.array([np.cos(np.pi/2),0.,0.,np.sin(np.pi/2)])]
for i in range(numparticles):           
    part = msmrd2.particle(D, Drot, positionList[i], orientationList[i])
    part.setType(i)
    pyPartlist.append(part)

In [ ]:
# Create list of particles that can be read from msmrd
# note the particles in this list will be independent from the python list.
partlist = msmrd2.integrators.particleList(pyPartlist)

In [ ]:
# Over-damped Langevin integrator definition
dt = 0.00001 #0.000005
seed = 0 # Seed = -1 used random device as seed
bodytype = 'rigidbody' # three rotational degrees of freedom
integrator = odLangevin(dt, seed, bodytype) 
integrator.setKbT(1.0)

In [ ]:
# Define boundary (choose either spherical or box)
radius = 3.0
sphereBoundary = msmrd2.sphere(radius,'reflective')
integrator.setBoundary(sphereBoundary)

In [ ]:
# Define Patchy Protein potential
sigma = 1.0
strength = 65
patchesCoordinates1 = [np.array([1.,0.,0.]), 
                       np.array([0.,1.,0.]),
                       np.array([-1.,0.,0.]),
                       np.array([0.,-1.,0.]),
                       np.array([0.,0.,1.]),
                       np.array([0.,0.,-1.])]
patchesCoordinates2 = [np.array([1.,0.,0.])]
potentialPatchyProtein = patchyProtein(sigma, strength, patchesCoordinates1, patchesCoordinates2)
integrator.setPairPotential(potentialPatchyProtein)

In [ ]:
# Integrate particle list and print only positions 
timesteps = 1000000 #50000000
stride = 1000 #1000
datafile  = open('../../data/vmd/patchyProteins.xyz', 'w')
for i in range(timesteps):
    if i%stride == 0:
        datafile.write(str(9) + '\n')
        datafile.write(str(0) + '\n')
    for j, part in enumerate(partlist):
        if i%stride == 0:
            if part.type == 0:
                v0 = part.position
                v1 = v0 + 0.4*sigma*quats.rotateVec(patchesCoordinates1[0], part.orientation)
                v2 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[1], part.orientation)
                v3 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[2], part.orientation)
                v4 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[3], part.orientation)
                v5 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[4], part.orientation)
                v6 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[5], part.orientation)
                datafile.write('type_0' + ' ' + ' '.join(map(str, v0)) + '\n')
                datafile.write('type_2' + ' ' + ' '.join(map(str, v1)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v2)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v3)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v4)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v5)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v6)) + '\n')
            if part.type == 1:
                v0 = part.position
                v1 = v0 + 0.4*sigma*quats.rotateVec(patchesCoordinates2[0], part.orientation)
                datafile.write('type_0' + ' ' + ' '.join(map(str, v0)) + '\n')
                datafile.write('type_2' + ' ' + ' '.join(map(str, v1)) + '\n')
    integrator.integrate(partlist)
    if i%5000 == 0:
        print("Percentage complete: ", 100*i/timesteps, "%", end="\r")
datafile.close()
# Generate TCL script to visualize with VMD
msmrdvis.generateTCL_patchyProteins(numparticles = numparticles, 
                                    outfname = "patchyProteins", 
                                    tclfname = "../../data/vmd/patchyProteins_2vmd.tcl")
print("Percentage complete: ", 100, " %")

To load the movie go to /data/vmd and run in a terminal "vmd -e patchyProteins_2vmd.tcl".

## In case VMD is not desired

In case VMD output is not desired, below we simply integrate the model and show the output directly.

In [ ]:
# Integrate particle list and print only positions of first and last particle in list
timesteps = 10
print('{:<10s}{:<15s}{:<40s}{:<40s}'.format("Iteration", "Time", "Position 1", "Position 2"))
for i in range(timesteps):
    print('{:<10d}{:<15f}{:<40s}{:<50s}'.format(i, integrator.clock, str(partlist[0].position), str(partlist[1].position)))
    integrator.integrate(partlist)